# Utilizando GeoPy

In [1]:
from geopy.geocoders import Nominatim

In [2]:
geolocator = Nominatim(user_agent="primeiro_teste_com_Nominatim", timeout=10)

In [3]:
location = geolocator.geocode("Rua São Bento, 407")

In [4]:
location

Location(Rua São Bento, Vila Alto do Cruzeiro, Praia Grande, Salvador, Microrregião de Salvador, Região Metropolitana de Salvador, Região Geográfica Intermediária de Salvador, Bahia, Região Nordeste, 40725-610, Brasil, (-12.8732009, -38.4717929, 0.0))

In [5]:
geocode = lambda query: geolocator.geocode("%s, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste" % query)

In [6]:
location = geocode("rua são bento, 405")

In [7]:
location

Location(Rua São Bento, Quitaúna, Osasco, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 06186-230, Brasil, (-23.5338883, -46.8164347, 0.0))

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv(f"dados/exemplo-1.csv")

In [11]:
df['endereco_sao_paulo'] = df['endereço'] +  ', São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste'

In [12]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [13]:
from tqdm import tqdm
tqdm.pandas()

/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
df['location'] = df['endereco_sao_paulo'].progress_apply(geocode)

 52%|█████▏    | 406/783 [12:20<11:15,  1.79s/it]

In [ ]:
df

In [ ]:
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
df[~df.point.isin([None])]

In [ ]:
df.to_csv("resultados/exemplo-1-nominatim.csv")

In [ ]:
import geopandas as gpd

In [ ]:
df_points = df[~df.point.isin([None])]

In [ ]:
df_points['x'] = df_points.apply(lambda row: row.point[1], axis=1)
df_points['y'] = df_points.apply(lambda row: row.point[0], axis=1)

In [ ]:
gdf = gpd.GeoDataFrame(df_points, geometry=gpd.points_from_xy(df_points.x, df_points.y))
# df_points.point[0][0]

In [ ]:
gdf

In [ ]:
# gdf.to_file('resultados/exemplo-1-nominatim.gpkg')
gdf[['id', 'endereço', 'geometry']].to_file('resultados/exemplo-1-nominatim.gpkg', driver="GPKG")